# Предсказываем ответ пользователя с помощью RNN (GRU)

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

import functools

### Подготовка данных
Данные хранятся в виде чатов:
* Одна строка - одна реплика
* Вопрос боту начинается с символа >
* Ответ бота начинается с символа <
* Чаты разделены строкой с текстом ===


In [2]:
# Загружаем набор чатов на которых будем обучаться
text = open('myMessages.txt', 'rb').read().decode(encoding='utf-8')
print ('Общее количество символов: {}'.format(len(text)))

# Составляем словарь символов
vocab = sorted(set(text))
print ('Уникальных символов: {}'.format(len(vocab)))

# Функции для преобразования текста в массив чисел и обратно
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

Общее количество символов: 13694221
Уникальных символов: 93


In [3]:
# Подгатавливаем обучающий датасет
seq_length = 100
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [4]:
for input_example, target_example in  dataset.take(1):
  print ('Вход:  ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Выход:', repr(''.join(idx2char[target_example.numpy()])))

Instructions for updating:
Colocations handled automatically by placer.
Вход:   '> я вот что поставила\n===\n< ок\n< откуда лиса?\n> не згаю, бвла в телефоне. плохая?\n< нет хорошая\n> а '
Выход: ' я вот что поставила\n===\n< ок\n< откуда лиса?\n> не згаю, бвла в телефоне. плохая?\n< нет хорошая\n> а ч'


In [7]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

SHUFFLE_BUFFER_SIZE = 10000
dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

### Модель сети

In [23]:
# Размер используемого словаря
vocab_size = len(vocab)
# Размерности сети
embedding_dim = 256
rnn_units = 1024

# Здесь можно попробовать различные варианты архитектуры сети. На текущий момент лучший вариант у LSTM.

#rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')
rnn = tf.keras.layers.CuDNNLSTM
#rnn = tf.keras.layers.CuDNNGRU

# Для ускорения работы на GPU можно использовать rnn = tf.keras.layers.CuDNNGRU, но такая сеть не может быть потом 
# использована для работы на CPU.

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  return tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
      rnn(rnn_units,
          return_sequences=True,
          recurrent_initializer='glorot_uniform',
          stateful=True),
      tf.keras.layers.Dense(vocab_size)
  ])

In [9]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23808     
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (64, None, 1024)          5251072   
_________________________________________________________________
dense (Dense)                (64, None, 93)            95325     
Total params: 5,370,205
Trainable params: 5,370,205
Non-trainable params: 0
_________________________________________________________________


In [10]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

### Обучение сети

In [11]:
# Настройка сохранения результатов
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [12]:
history = model.fit(dataset.repeat(), epochs=5, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/5
2138/2139 [============================>.] - ETA: 0s - loss: 1.7243WARNING:tensorflow:From /mnt/udata/anaconda/lib/python3.7/site-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
2139/2139 [==============================] - 172s 80ms/step - loss: 1.7241
Epoch 2/5
2139/2139 [==============================] - 175s 82ms/step - loss: 1.4423
Epoch 3/5
2139/2139 [==============================] - 171s 80ms/step - loss: 1.3940
Epoch 4/5
2139/2139 [==============================] - 173s 81ms/step - loss: 1.3664
Epoch 5/5
2139/2139 [==============================] - 173s 81ms/step - loss: 1.3470


### Загрузка обученой модели из файла

In [24]:
# Загружаем модель из checkpoint'а, при этом используем batch_size размера 1, чтобы можно было использовать 
# модель в режиме чата
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            23808     
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (1, None, 1024)           5251072   
_________________________________________________________________
dense_2 (Dense)              (1, None, 93)             95325     
Total params: 5,370,205
Trainable params: 5,370,205
Non-trainable params: 0
_________________________________________________________________


### Тестирование результатов работы

In [14]:
def generate_text(model, start_string, oneString, temperature):
  # Максимальное количество генерируемых символов
  num_generate = 200
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    if temperature > 0:
        predictions = predictions / temperature
    predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    c = idx2char[predicted_id]
    text_generated.append(c)
    if c == '\n' and oneString:
        break

  return (start_string + ''.join(text_generated))

#### Автоматическое завершение чатов

In [21]:
print(generate_text(model, "===\n> куда завтра сходим?\n< в ", True, 0.001))
print(generate_text(model, "===\n> во сколько?\n< в ", True, 0.001))
print(generate_text(model, "===\n> где ты сейчас?\n< в ", True, 0.001))
print(generate_text(model, "===\n> где ключи от машины?\n< в ", True, 0.001))

===
> куда завтра сходим?
< в магазин

===
> во сколько?
< в 8 начал

===
> где ты сейчас?
< в метро

===
> где ключи от машины?
< в машине



#### Чат-бот

In [45]:
dialog = u"===\n"
while(True):
  rq = input("> ")
  if rq == '':
        break;
  dialog += "> " + rq + "\n< "
  
  fullAns = generate_text(model, start_string=dialog, temperature=0.1, oneString = True)
  shortAns = fullAns[len(dialog):]
  print("< " + shortAns)
  dialog = fullAns

> ты сейчас где?
< в метро

> домой едешь?
< нет

> а куда?
< на дачу

> баг поправил?
< да

> отлично!
< ну вот посмотрим

> 


#### Генератор чатов

In [29]:
print(generate_text(model, "===", False, 0.25))

====
> как дела?
< норм
===
> как дела?
< да ничего
> ты где?
< в метро
> а что там?
< ну вот подумали в поликлинику
> а ты как?
< пока нет
< не знаю
< надо посмотреть
> а сейчас он сказал?
< нет
> ну вот как ты сегодня?
< ну пока не пойду
> ну вот ты там поедешь на дачу?
< нет
< собираюсь завтра с дедушкой и пойду
> ну вот посмотрим
> ну вот смотри какой с помощью калининграда
> ну вот почему не поймешь, что ты так думаешь, что вот подумал, что ты не поедешь :(
< ну вот ты так получил себя чувствую, что они там получается в доме как раз на полу в области до 10 лет
< ну да, ну вот почему не будет до этого в него надо было сказать что это тоже самое время получается
> ну, вот ты мне позвонил, он не помнится, что он не помогло было бы сделать с собой воду и спросила про нас на полчаса
< ну да
< ну надо было сказать что не получилось
> ну почему не поймут?
< ну я понимаю, что так не будет
> ну почему не надо?
> ну вот и пойду
> не пойду
< ну просто спросил, надо позвонить
< сейчас пойду
>